In [1]:
import requests
import json 
import plotly.express as px
import pandas as pd
import numpy as np

** Rouen Parking API**

+ Using this API to get parking informaiton : https://data.strasbourg.eu/explore/dataset/parkings/table/
+ Using this API to get real time information https://data.opendatasoft.com/explore/dataset/occupation-parkings-temps-reel%40eurometrostrasbourg/api/: 

In [2]:
Parking_info = requests.get("https://data.strasbourg.eu/api/explore/v2.1/catalog/datasets/parkings/records?limit=100&timezone=europe%2FParis") 
Parking_info

<Response [200]>

In [3]:
Dump_parking_info = json.dumps(Parking_info.json())
Loads_parking_info = json.loads(Dump_parking_info)  
parking_info = pd.json_normalize(Loads_parking_info['results'])
parking_info.head(2)

,city,friendlyurl,districtcode,description,address,types,name,idsurfs,zipcode,citycode,...,accessforelder,street,accessforwheelchair,accessfordeaf,normalizedalias,accessforblind,schedulelinkurl,hasurlschedule,position.lon,position.lat
0,Strasbourg,https://www.strasbourg.eu/lieu/-/entity/id/452...,SQ_116,Afin de faciliter le stationnement de votre vé...,4 Rue Catherine Kany 67000 Strasbourg France,Cat_22_02,Parking Rhénapark,2471_DEP_64,67000,C_482,...,0,4 Rue Catherine Kany,1,0,parking-rhenapark,0,{},0,7.792645,48.570974
1,Strasbourg,https://www.strasbourg.eu/lieu/-/entity/id/452...,SQ_112,Parking moyenne durée Reservation et location ...,4 rue Edmond Michelet 67000 Strasbourg France,Cat_22_02,Parking Danube Bleu,2472_DEP_65,67000,C_482,...,0,4 rue Edmond Michelet,1,0,parking-danube-bleu,0,{},0,7.766533,48.572834


**Real Time information**

In [4]:


from datetime import datetime 
import pytz 
UTC = pytz.utc
time_At_P = pytz.timezone('Europe/Paris') 
dt_time_P = datetime.now(time_At_P) 
print(dt_time_P.strftime("%Y-%m-%d %H:%M:%S"))

2024-06-28 16:20:56


In [5]:
Parking_reel_tine = requests.get("https://data.opendatasoft.com/api/explore/v2.1/catalog/datasets/occupation-parkings-temps-reel@eurometrostrasbourg/records?limit=100&timezone=europe%2FParis") 
Parking_reel_tine

<Response [200]>

In [7]:
Dump_RT_parking = json.dumps(Parking_reel_tine.json())
Loads_RT_parking = json.loads(Dump_RT_parking)  
RT_parking = pd.json_normalize(Loads_RT_parking['results'])

#Compute data
RT_parking["Occupee"] = RT_parking["total"]  - RT_parking["libre"] 
RT_parking["percentage_occupe"] = RT_parking["Occupee"]  / RT_parking["total"]  

RT_parking.head(3)

,nom_parking,idsurfs,ident,etat,etat_descriptif,total,libre,infousager,realtimestatus,infoappli,Occupee,percentage_occupe
0,Parking Gare Wodli,1382_DEP_8,10,1,Ouvert,955,390,390,GREEN,390,565,0.591623
1,Parking Zénith,1400_DEP_26,1,2,Fermé,3175,3140,0,BLACK,Fermé,35,0.011024
2,Parking Centre Opéra Broglie,1378_DEP_4,3,1,Ouvert,447,36,30,RED,36,411,0.919463


In [8]:
parking_info = parking_info[["name" , "position.lon",	"position.lat"]]
RT_parking = RT_parking.merge(parking_info, how='inner' , left_on='nom_parking' , right_on = 'name')

In [9]:
RT_parking
#RT_parking.loc[RT_parking['etat_descriptif'] == "Ouvert"]
RT_parking_ouvert = RT_parking.loc[RT_parking['etat_descriptif'] == "Ouvert"]
RT_parking_ferme = RT_parking.loc[RT_parking['etat_descriptif'] == "Fermé"]
RT_parking_not_available = RT_parking.loc[RT_parking['etat_descriptif'] == "frequentation temps reel indisponible"]

# Top 3 sites with bigger fullness

In [10]:
RT_parking_ouvert.sort_values(by='percentage_occupe', ascending=False).head(3)

,nom_parking,idsurfs,ident,etat,etat_descriptif,total,libre,infousager,realtimestatus,infoappli,Occupee,percentage_occupe,name,position.lon,position.lat
16,Parking Rotonde,1394_DEP_20,7,1,Ouvert,450,21,20,RED,21,429,0.953333,Parking Rotonde,7.726101,48.591226
2,Parking Centre Opéra Broglie,1378_DEP_4,3,1,Ouvert,447,36,30,RED,36,411,0.919463,Parking Centre Opéra Broglie,7.749275,48.585689
4,Parking Bateliers,1377_DEP_3,2,1,Ouvert,283,32,30,RED,32,251,0.886926,Parking Bateliers,7.756048,48.581002


## Average % occupation

In [11]:
round(sum(RT_parking["Occupee"]) / sum(RT_parking["total"]) ,3)

0.567

## Numbre place dispo/occupe

In [12]:
NBR_place_libre = sum(RT_parking["libre"])
NBR_place_occupe = sum(RT_parking["Occupee"])
nbr_place_total  = sum(RT_parking["total"])
print(NBR_place_libre,NBR_place_occupe , nbr_place_total)

6874 9008 15882


## Nombre parking close

In [13]:
#close_site = RT_parking_not_available["nom_parking"].unique()

close_site = RT_parking_ferme["nom_parking"].unique()
' ,'.join(close_site)

'Parking Zénith'

## Bar plot occupation


In [14]:
#import package
import plotly.io as pio

#And the following will do the trick:
pio.templates.default = "plotly"

In [15]:
fig = px.bar(RT_parking, x='nom_parking', y='percentage_occupe',color="realtimestatus",
             color_discrete_sequence=["orange", "black", "green", "red", "blue"] )
fig.update_layout(xaxis={'categoryorder':'total descending'}) 
fig.show()


In [16]:
a= RT_parking_ouvert.sort_values('percentage_occupe', ascending=False)['nom_parking'].to_list()

In [17]:
from plotly.subplots import make_subplots
fig = make_subplots(shared_yaxes=True, shared_xaxes=True)
fig.add_bar(x=RT_parking_ouvert['nom_parking'],y=RT_parking_ouvert['total'],opacity=0.6,width=0.95,name='total',hovertemplate='%{y}')
fig.add_bar(x=RT_parking_ouvert['nom_parking'],y=RT_parking_ouvert['Occupee'],width=0.95,name='Occupee')

fig.update_layout(barmode='overlay', 
                  title= "Occupation parking Strasbourg",
                  yaxis_title='CPUs', xaxis={'categoryorder':'array', 'categoryarray':a})

#fig.update_layout(barmode='stack', xaxis={'categoryorder':'array', 'categoryarray':a}) 

## Maps creation


In [18]:
RT_parking.head(3)

,nom_parking,idsurfs,ident,etat,etat_descriptif,total,libre,infousager,realtimestatus,infoappli,Occupee,percentage_occupe,name,position.lon,position.lat
0,Parking Gare Wodli,1382_DEP_8,10,1,Ouvert,955,390,390,GREEN,390,565,0.591623,Parking Gare Wodli,7.736771,48.587385
1,Parking Zénith,1400_DEP_26,1,2,Fermé,3175,3140,0,BLACK,Fermé,35,0.011024,Parking Zénith,7.686467,48.595335
2,Parking Centre Opéra Broglie,1378_DEP_4,3,1,Ouvert,447,36,30,RED,36,411,0.919463,Parking Centre Opéra Broglie,7.749275,48.585689


In [19]:
import viridis

In [20]:
fig = px.scatter_mapbox(RT_parking, 
                        lat=RT_parking['position.lat'],
                        lon=RT_parking['position.lon'], 
                        hover_name="nom_parking", 
                        size="etat" ,
                        #hover_data=["status" , "Bike available" , "Available space" ],
                        custom_data=['nom_parking', 'etat_descriptif', 'libre' , 'Occupee','percentage_occupe'] ,
                        color="percentage_occupe",
                        color_continuous_scale=px.colors.sequential.OrRd ,
                        opacity = 0.8,
                        zoom=11, 
                        height=450)
fig.update_layout(mapbox_style="carto-positron")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.update_layout(coloraxis_showscale=True)
fig.update_traces(
    hovertemplate = 
                "<b>%{customdata[0]}</b><br>"   +  
                "<b>Etat : </b> %{customdata[1]}<br>" + 
                "<b>Parking libre: </b> %{customdata[2]}<br>" +
                "<b>Parking occupe: </b> %{customdata[3]}<br>"  +
                "<b>Pourcentage occupation: </b> %{customdata[4]}<br>"  
                ) 
fig.show()

## Status parking

In [21]:
RT_parking
etat_data = RT_parking.groupby(['etat_descriptif'], as_index=False).size()
#DataFrame({'count' : RT_parking.groupby( [ "etat_descriptif"] ).size()}).reset_index()
etat_data["share_size"] = round((etat_data['size'] / etat_data['size'].sum()) ,2)
etat_data

,etat_descriptif,size,share_size
0,Fermé,1,0.04
1,Ouvert,20,0.74
2,frequentation temps reel indisponible,6,0.22


In [25]:
status_bar = px.bar(etat_data, y='etat_descriptif',
            x='size' ,
            width=600,
            height=250,
            orientation='h' ,
            color = 'size' ,
            color_continuous_scale=px.colors.sequential.OrRd[::-1])
status_bar.update_layout(yaxis={'categoryorder':'total ascending'},
                  margin=dict(l=20, r=20, t=20, b=20),) 
status_bar.update_layout(coloraxis_showscale=False)
status_bar.show()

In [27]:
#import sys
#print(sys.version)

3.11.2 (tags/v3.11.2:878ead1, Feb  7 2023, 16:38:35) [MSC v.1934 64 bit (AMD64)]


import sys
print(sys.version)